# Lab 9

## TODO:

- [ ] Linear classifier
  - [ ] Use training data, note MSE
  - [ ] Can we say the data is linearly separable?
  - [ ] Note MSE on test data
  - [ ] Does the model generalise?
- [ ] Logistic regression
  - [ ] Use training data, note MSE
  - [ ] Note MSE on test data
  - [ ] Record and explain findings
- [ ] (optional) Experiment with hyperparameters (maybe use random search)
  - [ ] learning rate
  - [ ] epochs
  - [ ] batch size
- [ ] (optional) Table or graph results

## Imports 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from scipy.stats import randint

## Data setup

In [5]:
np.random.seed(0)
os.environ["OMP_NUM_THREADS"] = "5"

df3 = pd.read_csv("Data/data3.csv")
df3

,failures,higher,studytime,Medu,Fedu,Dalc,age,reason_reputation,school,address,internet,G3
0,0,1,2,4,4,1,18,0,0,0,0,6
1,0,1,2,1,1,1,17,0,0,0,1,6
2,3,1,2,1,1,2,15,0,0,0,1,10
3,0,1,3,4,2,1,15,0,0,0,1,15
4,0,1,2,3,3,1,16,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1039,1,1,3,2,3,1,19,0,1,1,1,10
1040,0,1,2,3,1,1,18,0,1,0,1,16
1041,0,1,2,1,1,1,18,0,1,0,0,9
1042,0,1,1,3,1,3,17,0,1,0,1,10


In [6]:
df3_copy = df3.copy()
y = df3_copy.pop("G3")
y_binary = [0 if y_i > np.mean(y) else 1 for y_i in y] #making the target class into binary (over/under average score)
X = df3_copy
# display(X)
# print(y_binary)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, stratify=y_binary, random_state=42,)
X_train.shape

(835, 11)

Note: No need to scale, trees are not affected by different scales.

## Linear Classifier

In [10]:
linear_table = pd.DataFrame(columns=["params...","train accuracy","test accuracy", "MSE"])

In [ ]:
parameters = {
    "learning_rate": [1.0, 0.1, 0.01, 0.001],
    "max_iter": [10, 100, 1000],
    "batch":

}
linear_model = SGDClassifier(loss="squared_error")
linear_cv = GridSearchCV(linear_model, parameters, cv=10)
linear_cv.fit(X_train,y_train)

### Table

In [11]:
linear_table

,params...,train accuracy,test accuracy,MSE


## Logistic Regression

In [12]:
logistic_table = pd.DataFrame(columns=["params...","train accuracy","test accuracy", "MSE"])

In [14]:
logistic_model = SGDClassifier(loss="log", random_state=10)
logistic_cv = GridSearchCV(logistic_model, parameters, cv=10)
logistic_cv.fit(X_train,y_train)

,params...,train accuracy,test accuracy,MSE


### Table

In [13]:
logistic_table

,params...,train accuracy,test accuracy,MSE
